In [ ]:
import jax.numpy as jnp

import pandas as pd

import make_df

import matplotlib.pyplot as plt

In [ ]:
def fun(x):
    return (x[:,0]**2 + x[:,1] - 11)**2 / 800.0 + (x[:,0] + x[:,1]**2 -7)**2 / 800.0

In [ ]:
# name of the function
name = "him"
# directory where to save stuff
in_dir = "./sparse"

# How many function and derivative observations should be chosen
list_f_vals = [1, 5, 20, 50]
list_d_vals = [200, 400, 800, 1500]
# Dimension of datapoints

# optimizer type
optimizers = ["L-BFGS-B", "TNC", "SLSQP"]
iters_per_optimmizer = 10

# Grid on which to evaluate the function
bounds = jnp.array([[-5.0, -5.0], [5.0, 5.0]])
eval_grid = jnp.linspace(bounds[0], bounds[1],100).T
eval_grid = jnp.array(jnp.meshgrid(*eval_grid)).reshape(2,-1).T

# sparsification
sparse = True
subset_size = 0.5

In [ ]:
data = make_df.make_df(list_f_vals, list_d_vals, optimizers, in_dir, name, sparse, subset_size, fun, eval_grid)

In [ ]:
data.head(5)

In [ ]:
bfgs_data = data[data["opt"] == "SLSQP"]
# bfgs_data = data[data["opt"] == "L-BFGS-B"]

In [ ]:
colors = ["b", "r", "g", "cyan"]

fig, ax = plt.subplots(2, 3, figsize=(15, 10))

for i, optimizer in enumerate(optimizers):
    opt_data = data[data["opt"] == optimizer]
    for c,f in zip(colors, opt_data["f"].unique()):
        temp = opt_data[opt_data["f"] == f]
        dvals = temp["d"]

        mean_mse = temp["mean_mse"]
        min_mse = temp["min_mse"]
        max_mse = temp["max_mse"]
        ax[0,i].plot(dvals, mean_mse, color=c, ls="--", marker="x", label=f"#f-vals = {f}")
        ax[0,i].fill_between(dvals, min_mse.tolist(), max_mse.tolist(), color=c, alpha=0.2)
        ax[0,i].set_yscale("log")
        # ax[0,i].set_xscale("log")

        mean_tic = temp["mean_tic"]
        min_tic = temp["min_tic"]
        max_tic = temp["max_tic"]
        ax[1,i].plot(dvals, mean_tic, color=c, ls="--", marker="x", label=f"#f-vals = {f}")
        ax[1,i].fill_between(dvals, min_tic.tolist(), max_tic.tolist(), color=c, alpha=0.2)


    ax[0,i].grid()
    ax[1,i].grid()
    ax[0,i].legend()
    ax[1,i].legend()
    ax[1,i].set_xlabel("#d-vals")
    ax[0,0].set_ylabel("MSE")
    ax[1,0].set_ylabel("% Y inside conf.-interval")
    ax[0,i].set_title(optimizer)